In [ ]:
import streamlit as st
from PIL import Image
from pyzbar.pyzbar import decode
import json

st.set_page_config(page_title="QR Scanner", page_icon="📷", layout="centered")

# --- Function to Decode Image ---
def decode_img(img_path):
    img = Image.open(img_path)
    result = decode(img)
    if result:
        data = result[0].data.decode("utf-8")
        try:
            # Try to parse as JSON (if QR contains JSON)
            return json.loads(data)
        except:
            return {"Decoded Data": data}
    return None

# --- Page Navigation using session_state ---
if "page" not in st.session_state:
    st.session_state.page = "scanner"

def go_to_result(decoded_data):
    st.session_state.page = "result"
    st.session_state.decoded_data = decoded_data

# --- Page 1: Scanner Page ---
if st.session_state.page == "scanner":
    st.title("📷 QR Code Scanner")

    img_file = st.camera_input("Click below to take a photo")

    if img_file:
        img = Image.open(img_file)
        img_path = "test.png"
        img.save(img_path)
        st.success("✅ Photo captured successfully!")

        decoded = decode_img(img_path)
        if decoded:
            go_to_result(decoded)
        else:
            st.error("⚠️ No QR code detected. Try again!")

# --- Page 2: Result Page ---
elif st.session_state.page == "result":
    st.title("✅ Decoded Information")

    decoded = st.session_state.get("decoded_data", {})
    st.json(decoded)

    if st.button("🔙 Scan Another"):
        st.session_state.page = "scanner"


In [ ]:
uploaded_file = st.file_uploader("Upload Image", type=["jpg", "jpeg", "png"])

if uploaded_file is not None:
    # Create save folder if it doesn't exist
    save_dir = "uploaded_images"
    os.makedirs(save_dir, exist_ok=True)
    
    # Define save path
    save_path = os.path.join(save_dir, uploaded_file.name)
    
    # Save image automatically
    image = Image.open(uploaded_file)
    image.save(save_path)
    
    st.success(f"✅ Image saved successfully to: {save_path}")
else:
    st.info("👆 Upload an image to auto-save it.")

In [ ]:
elif st.session_state.page == "result":
    st.title("✅ Decoded Information")

    decoded = st.session_state.get("decoded_data", {})

    if decoded:
        # --- Display the image in the center ---
        if "image" in decoded and decoded["image"]:
            st.markdown(
                f"""
                <div style="display: flex; justify-content: center; margin-top: 20px;">
                    <img src="{decoded['image']}" alt="User Image"
                        style="width: 250px; height: 250px; border-radius: 15px; object-fit: cover; box-shadow: 0 0 15px rgba(0,0,0,0.6);">
                </div>
                """,
                unsafe_allow_html=True
            )
            st.markdown("<br>", unsafe_allow_html=True)

        # --- Display the rest of the details below the image ---
        st.subheader("📄 Details")
        details_to_show = {k: v for k, v in decoded.items() if k != "image"}

        # Use a nice bordered container for details
        st.markdown(
            "<div style='background-color:#1a1b1e; padding: 20px; border-radius: 12px; box-shadow: 0 0 15px rgba(59,130,246,0.3);'>",
            unsafe_allow_html=True,
        )

        for key, value in details_to_show.items():
            st.markdown(f"<p style='font-size:18px;'><b>{key.title()}:</b> {value}</p>", unsafe_allow_html=True)

        st.markdown("</div>", unsafe_allow_html=True)

    else:
        st.warning("⚠️ No decoded data found!")

    # --- Button to scan another QR ---
    st.markdown("<br>", unsafe_allow_html=True)
    if st.button("🔙 Scan Another"):
        st.session_state.page = "scanner"
        st.rerun()


In [ ]:
elif st.session_state.page == "result":
    st.title("✅ Decoded Information")

    decoded = st.session_state.get("decoded_data", {})
    st.json(decoded)

    if st.button("🔙 Scan Another"):
        st.session_state.page = "scanner"

In [ ]:
import streamlit as st

# ---- Initialize session state ----
if "page" not in st.session_state:
    st.session_state.page = "scanner"

# ---- SCANNER PAGE ----
if st.session_state.page == "scanner":
    st.title("📷 QR Scanner")
    st.write("Scan a QR or click below to view stored info.")

    # Example data (you can replace this with decoded QR data)
    st.session_state.decoded_data = {
        "Name": "Prashanth",
        "Blood Group": "B+",
        "Emergency Contact": "95991576744"
    }

    # 🔘 Button to navigate to Result page
    if st.button("👁️ View Info", use_container_width=True):
        st.session_state.page = "result"
        st.experimental_rerun()  # or st.rerun() in latest Streamlit

# ---- RESULT PAGE ----
elif st.session_state.page == "result":
    st.title("✅ Patient Information")

    decoded = st.session_state.get("decoded_data", {})
    if decoded:
        col1, col2, col3 = st.columns([1, 2, 1])
        with col2:
            # If you have a saved image path
            image_path = "test.png"
            if image_path:
                st.image(image_path, caption="🩺 Patient Photo", use_container_width=True)
            st.write("### 👇 Patient Details")
            st.json(decoded)
    else:
        st.warning("⚠️ No patient data found!")

    if st.button("🔙 Back to Scanner", use_container_width=True):
        st.session_state.page = "scanner"
        st.experimental_rerun()


In [ ]:
elif st.session_state.page == "scanner":
    st.title("📷 Scanner")

    # Run scan() only if data not already captured
    if "decoded_data" not in st.session_state:
        decoded = scan()
        if decoded is not None:
            st.session_state.decoded_data = decoded
            st.success("✅ QR decoded successfully!")
    else:
        st.info("✅ QR data already captured — click below to view info.")

    # Show the View Info button when decoded data exists
    if "decoded_data" in st.session_state:
        if st.button("View Info", use_container_width=True):
            st.session_state.page = "result"
            st.rerun()


In [ ]:
import streamlit as st
import qrcode
import json
import os
from PIL import Image

def add_info():
    # ---- Page Setup ----
    #st.set_page_config(page_title="Life Saving QR Generator", layout="wide")

    # ---- Custom CSS ----
    st.markdown("""
    <style>
    body {
        background: linear-gradient(135deg, #000000, #1a0000, #330000);
        font-family: 'Poppins', sans-serif;
        color: #fff;
    }

    /* Center container */
    .main-title {
        text-align: center;
        font-size: 2.8rem;
        color: #ff073a;
        text-shadow: 0 0 20px #ff073a, 0 0 50px #ff073a;
        font-weight: 800;
        margin-top: 15px;
        animation: glowPulse 2s ease-in-out infinite alternate;
    }

    @keyframes glowPulse {
        from { text-shadow: 0 0 10px #ff073a; }
        to { text-shadow: 0 0 40px #ff073a, 0 0 80px #ff073a; }
    }

    .stTextInput > div > div > input {
        background-color: rgba(255, 255, 255, 0.1);
        color: white;
        border: 1px solid #ff073a;
        border-radius: 10px;
        padding: 10px;
        font-size: 1rem;
        transition: all 0.3s ease-in-out;
    }

    .stTextInput > div > div > input:focus {
        border: 1px solid #ff5075;
        box-shadow: 0 0 20px #ff073a;
    }

    /* File uploader */
    .stFileUploader > div > div {
        background-color: rgba(255, 7, 58, 0.05);
        border: 1px dashed #ff073a;
        border-radius: 10px;
    }

    /* Buttons */
    .stButton > button {
        background: linear-gradient(90deg, #ff073a, #ff3366);
        color: white;
        font-weight: 600;
        padding: 12px 25px;
        border-radius: 10px;
        font-size: 1.1rem;
        border: none;
        box-shadow: 0 0 20px rgba(255, 7, 58, 0.6);
        transition: all 0.3s ease-in-out;
    }

    .stButton > button:hover {
        background: linear-gradient(90deg, #ff5075, #ff073a);
        transform: scale(1.05);
        box-shadow: 0 0 35px rgba(255, 7, 58, 0.9);
    }

    /* Success & warning messages */
    .stSuccess, .stWarning, .stInfo {
        background-color: rgba(255, 7, 58, 0.1);
        border-left: 4px solid #ff073a;
        border-radius: 6px;
        color: white;
    }

    /* QR Display */
    .qr-container {
        text-align: center;
        margin-top: 25px;
        animation: fadeIn 2s ease;
    }

    .qr-container img {
        border-radius: 12px;
        box-shadow: 0 0 40px #ff073a;
        animation: pulseQR 2s ease-in-out infinite alternate;
    }

    @keyframes pulseQR {
        from { box-shadow: 0 0 20px #ff073a; }
        to { box-shadow: 0 0 60px #ff073a; }
    }

    @keyframes fadeIn {
        from { opacity: 0; transform: translateY(10px); }
        to { opacity: 1; transform: translateY(0); }
    }

    /* Divider line */
    hr {
        border: none;
        height: 1px;
        background: linear-gradient(90deg, transparent, #ff073a, transparent);
        margin: 20px 0;
    }

    </style>
    """, unsafe_allow_html=True)

    # ---- Page Title ----
    st.markdown("<h1 class='main-title'>🩺 Life Saving QR Code Generator</h1>", unsafe_allow_html=True)
    st.markdown("<hr>", unsafe_allow_html=True)

    # ---- User Inputs ----
    name = st.text_input("Name", placeholder="e.g. Prashanth")
    phone = st.text_input("User Phone Number", placeholder="e.g. 9876543210", max_chars=10)
    Eme_contact1 = st.text_input("Emergency Contact Number 1", placeholder="e.g. 9876543210", max_chars=10)
    Eme_contact2 = st.text_input("Emergency Contact Number 2", placeholder="e.g. 9876543210", max_chars=10)
    Eme_contact3 = st.text_input("Emergency Contact Number 3", placeholder="e.g. 9876543210", max_chars=10)
    blood_group = st.text_input("Blood Group", placeholder="e.g. B+")
    medical = st.text_input("Medical Condition (if any)", placeholder="e.g. Diabetes", value="NO")
    ad = st.text_input("Aadhaar Number", placeholder="e.g. 6555 xxxx xxxx 2412")
    insurance = st.text_input("Insurance Number", placeholder="e.g. 890xxxxxxxx", value="NO")
    dob = st.text_input("Date of Birth", placeholder="e.g. 15/09/2004")

    uploaded_file = st.file_uploader("Upload Image", type=["jpg", "jpeg", "png"])

    if uploaded_file is not None and phone.isdigit() and len(phone) == 10:
        save_dir = "data"
        os.makedirs(save_dir, exist_ok=True)
        save_path = os.path.join(save_dir, f"{phone}.jpg")
        image = Image.open(uploaded_file)
        image.save(save_path, format="JPEG")
        st.success(f"✅ Image saved successfully to: {save_path}")
    elif uploaded_file is not None:
        st.warning("⚠️ Invalid phone number — please enter a valid 10-digit number.")

    # ---- QR Code Generation ----
    button = st.button("Generate QR Code")
    if button:
        if all([name, phone, Eme_contact1, Eme_contact2, Eme_contact3, blood_group, medical, ad, insurance, dob]) and uploaded_file is not None:
            patient_info = {
                "Name": name,
                "Phone": phone,
                "Emergency_contact1": Eme_contact1,
                "Emergency_contact2": Eme_contact2,
                "Emergency_contact3": Eme_contact3,
                "Blood_group": blood_group,
                "Medical_condition": medical,
                "Aadhaar": ad,
                "Insurance": insurance,
                "DOB": dob
            }

            data = json.dumps(patient_info)
            qr = qrcode.make(data)
            qr.save("patient_qr.png")

            st.markdown("<div class='qr-container'>", unsafe_allow_html=True)
            st.image("patient_qr.png", caption=f"{name}'s QR Code", use_container_width=False)
            st.markdown("</div>", unsafe_allow_html=True)

            with open("patient_qr.png", "rb") as file:
                st.download_button(
                    label="⬇️ Download QR Code",
                    data=file,
                    file_name=f"{name}_QR.png",
                    mime="image/png"
                )
        else:
            st.warning("⚠️ Please fill in all fields and upload an image before generating the QR Code.")


In [ ]:
import streamlit as st
import qrcode
import json
import os
from PIL import Image

def add_info():
    # ---- Custom CSS ----
    st.markdown("""
    <style>
    /* === Global Styling === */
    body {
        background: radial-gradient(circle at top left, #0d0d0d, #1a0000, #330000);
        color: white;
        font-family: 'Poppins', sans-serif;
    }

    .main-title {
        text-align: center;
        font-size: 2.8rem;
        color: #ff073a;
        text-shadow: 0 0 25px #ff073a, 0 0 60px #ff073a;
        font-weight: 800;
        margin-top: 10px;
        animation: titlePulse 2s ease-in-out infinite alternate;
    }

    @keyframes titlePulse {
        from { text-shadow: 0 0 15px #ff073a; }
        to { text-shadow: 0 0 45px #ff073a, 0 0 90px #ff073a; }
    }

    hr {
        border: none;
        height: 2px;
        background: linear-gradient(90deg, transparent, #ff073a, transparent);
        margin: 20px 0;
        animation: linePulse 3s linear infinite;
    }

    @keyframes linePulse {
        0% { opacity: 0.3; }
        50% { opacity: 1; }
        100% { opacity: 0.3; }
    }

    /* === Input Fields === */
    .stTextInput > div > div > input {
        background-color: rgba(255, 7, 58, 0.05);
        border: 1px solid rgba(255, 7, 58, 0.5);
        border-radius: 12px;
        padding: 12px;
        color: #fff;
        font-size: 1rem;
        transition: all 0.4s ease-in-out;
        box-shadow: 0 0 10px rgba(255, 7, 58, 0.1);
    }

    .stTextInput > div > div > input:focus {
        background-color: rgba(255, 7, 58, 0.1);
        border-color: #ff073a;
        box-shadow: 0 0 25px rgba(255, 7, 58, 0.7);
        transform: scale(1.02);
    }

    .stTextInput label {
        color: #ff6b81 !important;
        font-weight: 600;
        text-shadow: 0 0 8px rgba(255, 7, 58, 0.4);
    }

    /* === File Uploader === */
    .stFileUploader > div > div {
        background-color: rgba(255, 7, 58, 0.05);
        border: 1px dashed #ff073a;
        border-radius: 10px;
        transition: all 0.3s ease;
    }
    .stFileUploader > div > div:hover {
        box-shadow: 0 0 20px rgba(255, 7, 58, 0.7);
    }

    /* === Buttons === */
    .stButton > button {
        background: linear-gradient(90deg, #ff073a, #ff4f7a);
        color: white;
        font-weight: 600;
        padding: 12px 28px;
        border-radius: 10px;
        font-size: 1.1rem;
        border: none;
        box-shadow: 0 0 25px rgba(255, 7, 58, 0.7);
        transition: all 0.3s ease;
    }

    .stButton > button:hover {
        transform: scale(1.08);
        background: linear-gradient(90deg, #ff5075, #ff073a);
        box-shadow: 0 0 45px rgba(255, 7, 58, 0.9);
    }

    /* === QR Code Display === */
    .qr-container {
        text-align: center;
        margin-top: 30px;
        animation: fadeIn 1.5s ease;
    }

    .qr-container img {
        border-radius: 12px;
        box-shadow: 0 0 35px #ff073a;
        animation: pulseQR 2s ease-in-out infinite alternate;
    }

    @keyframes pulseQR {
        from { box-shadow: 0 0 25px #ff073a; }
        to { box-shadow: 0 0 65px #ff073a; }
    }

    /* === Heartbeat Line Animation === */
    .heartbeat-line {
        position: relative;
        width: 100%;
        height: 2px;
        background: linear-gradient(90deg, transparent, #ff073a, transparent);
        overflow: hidden;
        margin: 30px 0;
    }

    .heartbeat-line::before {
        content: '';
        position: absolute;
        left: -10%;
        width: 20%;
        height: 100%;
        background: #ff073a;
        animation: heartbeatMove 2.5s infinite ease-in-out;
    }

    @keyframes heartbeatMove {
        0% { left: -20%; opacity: 0; }
        25% { left: 25%; opacity: 1; }
        50% { left: 50%; opacity: 0.8; }
        75% { left: 75%; opacity: 1; }
        100% { left: 120%; opacity: 0; }
    }

    </style>
    """, unsafe_allow_html=True)

    # ---- Page Content ----
    st.markdown("<h1 class='main-title'>🩺 Life Saving QR Code Generator</h1>", unsafe_allow_html=True)
    st.markdown("<div class='heartbeat-line'></div>", unsafe_allow_html=True)

    # ---- Input Fields ----
    name = st.text_input("Name", placeholder="e.g. Prashanth")
    phone = st.text_input("User Phone Number", placeholder="e.g. 9876543210", max_chars=10)
    Eme_contact1 = st.text_input("Emergency Contact 1", placeholder="e.g. 9876543210", max_chars=10)
    Eme_contact2 = st.text_input("Emergency Contact 2", placeholder="e.g. 9876543210", max_chars=10)
    Eme_contact3 = st.text_input("Emergency Contact 3", placeholder="e.g. 9876543210", max_chars=10)
    blood_group = st.text_input("Blood Group", placeholder="e.g. B+")
    medical = st.text_input("Medical Condition (if any)", placeholder="e.g. Diabetes", value="NO")
    ad = st.text_input("Aadhaar Number", placeholder="e.g. 6555 xxxx xxxx 2412")
    insurance = st.text_input("Insurance Number", placeholder="e.g. 890xxxxxxxx", value="NO")
    dob = st.text_input("Date of Birth", placeholder="e.g. 15/09/2004")

    uploaded_file = st.file_uploader("Upload Image", type=["jpg", "jpeg", "png"])

    if uploaded_file is not None and phone.isdigit() and len(phone) == 10:
        save_dir = "data"
        os.makedirs(save_dir, exist_ok=True)
        save_path = os.path.join(save_dir, f"{phone}.jpg")
        image = Image.open(uploaded_file)
        image.save(save_path, format="JPEG")
        st.success(f"✅ Image saved successfully to: {save_path}")
    elif uploaded_file is not None:
        st.warning("⚠️ Invalid phone number — please enter a valid 10-digit number.")

    # ---- Generate QR ----
    if st.button("Generate QR Code"):
        if all([name, phone, Eme_contact1, Eme_contact2, Eme_contact3, blood_group, medical, ad, insurance, dob]) and uploaded_file is not None:
            patient_info = {
                "Name": name,
                "Phone": phone,
                "Emergency_contact1": Eme_contact1,
                "Emergency_contact2": Eme_contact2,
                "Emergency_contact3": Eme_contact3,
                "Blood_group": blood_group,
                "Medical_condition": medical,
                "Aadhaar": ad,
                "Insurance": insurance,
                "DOB": dob
            }

            data = json.dumps(patient_info)
            qr = qrcode.make(data)
            qr.save("patient_qr.png")

            st.markdown("<div class='qr-container'>", unsafe_allow_html=True)
            st.image("patient_qr.png", caption=f"{name}'s QR Code", use_container_width=False)
            st.markdown("</div>", unsafe_allow_html=True)

            with open("patient_qr.png", "rb") as file:
                st.download_button(
                    label="⬇️ Download QR Code",
                    data=file,
                    file_name=f"{name}_QR.png",
                    mime="image/png"
                )
        else:
            st.warning("⚠️ Please fill in all fields and upload an image before generating the QR Code.")


In [ ]:
import streamlit as st
import qrcode
import json
import os
from PIL import Image

def add_info():
    # ---- Custom CSS ----
    st.markdown("""
    <style>
    /* === Global Styling === */
    body {
        background: radial-gradient(circle at top left, #0d0d0d, #1a0000, #330000);
        color: white;
        font-family: 'Poppins', sans-serif;
    }

    /* === Title === */
    .main-title {
        text-align: center;
        font-size: 2.8rem;
        color: #ff073a;
        text-shadow: 0 0 25px #ff073a, 0 0 60px #ff073a;
        font-weight: 800;
        margin-top: 10px;
        animation: titlePulse 2s ease-in-out infinite alternate;
    }

    @keyframes titlePulse {
        from { text-shadow: 0 0 15px #ff073a; }
        to { text-shadow: 0 0 45px #ff073a, 0 0 90px #ff073a; }
    }

    hr {
        border: none;
        height: 2px;
        background: linear-gradient(90deg, transparent, #ff073a, transparent);
        margin: 20px 0;
        animation: linePulse 3s linear infinite;
    }

    @keyframes linePulse {
        0% { opacity: 0.3; }
        50% { opacity: 1; }
        100% { opacity: 0.3; }
    }

    /* === Heartbeat Line === */
    .heartbeat-line {
        position: relative;
        width: 100%;
        height: 2px;
        background: linear-gradient(90deg, transparent, #ff073a, transparent);
        overflow: hidden;
        margin: 25px 0;
    }

    .heartbeat-line::before {
        content: '';
        position: absolute;
        left: -10%;
        width: 20%;
        height: 100%;
        background: #ff073a;
        animation: heartbeatMove 2.5s infinite ease-in-out;
    }

    @keyframes heartbeatMove {
        0% { left: -20%; opacity: 0; }
        25% { left: 25%; opacity: 1; }
        50% { left: 50%; opacity: 0.8; }
        75% { left: 75%; opacity: 1; }
        100% { left: 120%; opacity: 0; }
    }

    /* === Input Fields (Upgraded) === */
    .stTextInput > div > div > input {
        background-color: rgba(255, 7, 58, 0.08);
        border: 2px solid rgba(255, 7, 58, 0.4);
        border-radius: 12px;
        padding: 12px 16px;
        color: #ffffff;
        font-size: 1rem;
        font-weight: 500;
        transition: all 0.4s ease-in-out;
        box-shadow: 0 0 8px rgba(255, 7, 58, 0.2);
        backdrop-filter: blur(6px);
    }

    /* Glow when focused */
    .stTextInput > div > div > input:focus {
        border-color: #ff073a;
        box-shadow: 0 0 25px #ff073a, inset 0 0 10px rgba(255, 7, 58, 0.5);
        background-color: rgba(255, 7, 58, 0.12);
        transform: scale(1.03);
    }

    /* Subtle glowing border animation */
    .stTextInput > div > div > input:hover {
        box-shadow: 0 0 15px rgba(255, 7, 58, 0.6);
        transition: 0.3s;
    }

    /* Animated placeholder */
    .stTextInput > div > div > input::placeholder {
        color: rgba(255, 200, 200, 0.5);
        font-style: italic;
        transition: color 0.3s ease;
    }

    .stTextInput > div > div > input:focus::placeholder {
        color: rgba(255, 7, 58, 0.7);
    }

    /* Label text */
    .stTextInput label {
        color: #ff6b81 !important;
        font-weight: 600;
        text-shadow: 0 0 8px rgba(255, 7, 58, 0.4);
        letter-spacing: 0.3px;
    }

    /* === File Uploader === */
    .stFileUploader > div > div {
        background-color: rgba(255, 7, 58, 0.05);
        border: 1px dashed #ff073a;
        border-radius: 10px;
        transition: all 0.3s ease;
    }

    .stFileUploader > div > div:hover {
        box-shadow: 0 0 20px rgba(255, 7, 58, 0.7);
    }

    /* === Buttons === */
    .stButton > button {
        background: linear-gradient(90deg, #ff073a, #ff4f7a);
        color: white;
        font-weight: 600;
        padding: 12px 28px;
        border-radius: 10px;
        font-size: 1.1rem;
        border: none;
        box-shadow: 0 0 25px rgba(255, 7, 58, 0.7);
        transition: all 0.3s ease;
    }

    .stButton > button:hover {
        transform: scale(1.08);
        background: linear-gradient(90deg, #ff5075, #ff073a);
        box-shadow: 0 0 45px rgba(255, 7, 58, 0.9);
    }

    /* === QR Code Display === */
    .qr-container {
        text-align: center;
        margin-top: 30px;
        animation: fadeIn 1.5s ease;
    }

    .qr-container img {
        border-radius: 12px;
        box-shadow: 0 0 35px #ff073a;
        animation: pulseQR 2s ease-in-out infinite alternate;
    }

    @keyframes pulseQR {
        from { box-shadow: 0 0 25px #ff073a; }
        to { box-shadow: 0 0 65px #ff073a; }
    }

    </style>
    """, unsafe_allow_html=True)

    # ---- Page Header ----
    st.markdown("<h1 class='main-title'>🩺 Life Saving QR Code Generator</h1>", unsafe_allow_html=True)
    st.markdown("<div class='heartbeat-line'></div>", unsafe_allow_html=True)

    # ---- Input Fields ----
    name = st.text_input("Name", placeholder="e.g. Prashanth")
    phone = st.text_input("User Phone Number", placeholder="e.g. 9876543210", max_chars=10)
    Eme_contact1 = st.text_input("Emergency Contact 1", placeholder="e.g. 9876543210", max_chars=10)
    Eme_contact2 = st.text_input("Emergency Contact 2", placeholder="e.g. 9876543210", max_chars=10)
    Eme_contact3 = st.text_input("Emergency Contact 3", placeholder="e.g. 9876543210", max_chars=10)
    blood_group = st.text_input("Blood Group", placeholder="e.g. B+")
    medical = st.text_input("Medical Condition (if any)", placeholder="e.g. Diabetes", value="NO")
    ad = st.text_input("Aadhaar Number", placeholder="e.g. 6555 xxxx xxxx 2412")
    insurance = st.text_input("Insurance Number", placeholder="e.g. 890xxxxxxxx", value="NO")
    dob = st.text_input("Date of Birth", placeholder="e.g. 15/09/2004")

    uploaded_file = st.file_uploader("Upload Image", type=["jpg", "jpeg", "png"])

    if uploaded_file is not None and phone.isdigit() and len(phone) == 10:
        save_dir = "data"
        os.makedirs(save_dir, exist_ok=True)
        save_path = os.path.join(save_dir, f"{phone}.jpg")
        image = Image.open(uploaded_file)
        image.save(save_path, format="JPEG")
        st.success(f"✅ Image saved successfully to: {save_path}")
    elif uploaded_file is not None:
        st.warning("⚠️ Invalid phone number — please enter a valid 10-digit number.")

    # ---- Generate QR ----
    if st.button("Generate QR Code"):
        if all([name, phone, Eme_contact1, Eme_contact2, Eme_contact3, blood_group, medical, ad, insurance, dob]) and uploaded_file is not None:
            patient_info = {
                "Name": name,
                "Phone": phone,
                "Emergency_contact1": Eme_contact1,
                "Emergency_contact2": Eme_contact2,
                "Emergency_contact3": Eme_contact3,
                "Blood_group": blood_group,
                "Medical_condition": medical,
                "Aadhaar": ad,
                "Insurance": insurance,
                "DOB": dob
            }

            data = json.dumps(patient_info)
            qr = qrcode.make(data)
            qr.save("patient_qr.png")

            st.markdown("<div class='qr-container'>", unsafe_allow_html=True)
            st.image("patient_qr.png", caption=f"{name}'s QR Code", use_container_width=False)
            st.markdown("</div>", unsafe_allow_html=True)

            with open("patient_qr.png", "rb") as file:
                st.download_button(
                    label="⬇️ Download QR Code",
                    data=file,
                    file_name=f"{name}_QR.png",
                    mime="image/png"
                )
        else:
            st.warning("⚠️ Please fill in all fields and upload an image before generating the QR Code.")


In [ ]:
import streamlit as st
import qrcode
import json
import os
from PIL import Image

def add_info():
    # ---- Custom CSS ----
    st.markdown("""
    <style>
    /* === Global Styling === */
    body {
        background: radial-gradient(circle at top left, #0d0d0d, #1a0000, #330000);
        color: white;
        font-family: 'Poppins', sans-serif;
    }

    /* === Title === */
    .main-title {
        text-align: center;
        font-size: 2.8rem;
        color: #ff073a;
        text-shadow: 0 0 25px #ff073a, 0 0 60px #ff073a;
        font-weight: 800;
        margin-top: 10px;
        animation: titlePulse 2s ease-in-out infinite alternate;
    }

    @keyframes titlePulse {
        from { text-shadow: 0 0 15px #ff073a; }
        to { text-shadow: 0 0 45px #ff073a, 0 0 90px #ff073a; }
    }

    /* === Heartbeat Line === */
    .heartbeat-line {
        position: relative;
        width: 100%;
        height: 2px;
        background: linear-gradient(90deg, transparent, #ff073a, transparent);
        overflow: hidden;
        margin: 25px 0;
    }

    .heartbeat-line::before {
        content: '';
        position: absolute;
        left: -10%;
        width: 20%;
        height: 100%;
        background: #ff073a;
        animation: heartbeatMove 2.5s infinite ease-in-out;
    }

    @keyframes heartbeatMove {
        0% { left: -20%; opacity: 0; }
        25% { left: 25%; opacity: 1; }
        50% { left: 50%; opacity: 0.8; }
        75% { left: 75%; opacity: 1; }
        100% { left: 120%; opacity: 0; }
    }

    /* === Input Fields (Advanced Neon Style) === */
    .stTextInput > div > div > input {
        background-color: rgba(255, 7, 58, 0.08);
        border: 2px solid rgba(255, 7, 58, 0.5);
        border-radius: 14px;
        padding: 12px 18px;
        color: #fff;
        font-size: 1rem;
        font-weight: 500;
        box-shadow: 0 0 8px rgba(255, 7, 58, 0.2);
        transition: all 0.4s ease;
        backdrop-filter: blur(6px);
        animation: subtlePulse 3s infinite ease-in-out;
    }

    /* Subtle border pulsing animation */
    @keyframes subtlePulse {
        0%, 100% { box-shadow: 0 0 10px rgba(255, 7, 58, 0.3); }
        50% { box-shadow: 0 0 25px rgba(255, 7, 58, 0.7); }
    }

    /* Focus animation */
    .stTextInput > div > div > input:focus {
        border-color: #ff073a;
        box-shadow: 0 0 30px #ff073a, inset 0 0 15px rgba(255, 7, 58, 0.6);
        background-color: rgba(255, 7, 58, 0.12);
        transform: scale(1.03);
        transition: all 0.3s ease;
    }

    /* Placeholder styling */
    .stTextInput > div > div > input::placeholder {
        color: rgba(255, 200, 200, 0.6);
        font-style: italic;
        transition: color 0.3s ease;
    }

    .stTextInput > div > div > input:focus::placeholder {
        color: rgba(255, 7, 58, 0.8);
    }

    /* === Bright Glowing Label === */
    .stTextInput label {
        color: #ff4f7a !important;
        font-weight: 700;
        font-size: 1.05rem;
        letter-spacing: 0.5px;
        text-shadow: 0 0 10px #ff073a, 0 0 25px rgba(255, 7, 58, 0.7);
        animation: glowLabel 2s ease-in-out infinite alternate;
    }

    @keyframes glowLabel {
        from { text-shadow: 0 0 10px #ff073a; }
        to { text-shadow: 0 0 25px #ff5075, 0 0 40px #ff073a; }
    }

    /* === File Uploader === */
    .stFileUploader > div > div {
        background-color: rgba(255, 7, 58, 0.05);
        border: 1px dashed #ff073a;
        border-radius: 10px;
        transition: all 0.3s ease;
    }

    .stFileUploader > div > div:hover {
        box-shadow: 0 0 20px rgba(255, 7, 58, 0.7);
    }

    /* === Buttons === */
    .stButton > button {
        background: linear-gradient(90deg, #ff073a, #ff4f7a);
        color: white;
        font-weight: 600;
        padding: 12px 28px;
        border-radius: 10px;
        font-size: 1.1rem;
        border: none;
        box-shadow: 0 0 25px rgba(255, 7, 58, 0.7);
        transition: all 0.3s ease;
    }

    .stButton > button:hover {
        transform: scale(1.08);
        background: linear-gradient(90deg, #ff5075, #ff073a);
        box-shadow: 0 0 45px rgba(255, 7, 58, 0.9);
    }

    /* === QR Code Display === */
    .qr-container {
        text-align: center;
        margin-top: 30px;
        animation: fadeIn 1.5s ease;
    }

    .qr-container img {
        border-radius: 12px;
        box-shadow: 0 0 35px #ff073a;
        animation: pulseQR 2s ease-in-out infinite alternate;
    }

    @keyframes pulseQR {
        from { box-shadow: 0 0 25px #ff073a; }
        to { box-shadow: 0 0 65px #ff073a; }
    }
    </style>
    """, unsafe_allow_html=True)

    # ---- Header ----
    st.markdown("<h1 class='main-title'>🩺 Life Saving QR Code Generator</h1>", unsafe_allow_html=True)
    st.markdown("<div class='heartbeat-line'></div>", unsafe_allow_html=True)

    # ---- Input Fields ----
    name = st.text_input("Name", placeholder="e.g. Prashanth")
    phone = st.text_input("User Phone Number", placeholder="e.g. 9876543210", max_chars=10)
    Eme_contact1 = st.text_input("Emergency Contact 1", placeholder="e.g. 9876543210", max_chars=10)
    Eme_contact2 = st.text_input("Emergency Contact 2", placeholder="e.g. 9876543210", max_chars=10)
    Eme_contact3 = st.text_input("Emergency Contact 3", placeholder="e.g. 9876543210", max_chars=10)
    blood_group = st.text_input("Blood Group", placeholder="e.g. B+")
    medical = st.text_input("Medical Condition (if any)", placeholder="e.g. Diabetes", value="NO")
    ad = st.text_input("Aadhaar Number", placeholder="e.g. 6555 xxxx xxxx 2412")
    insurance = st.text_input("Insurance Number", placeholder="e.g. 890xxxxxxxx", value="NO")
    dob = st.text_input("Date of Birth", placeholder="e.g. 15/09/2004")

    uploaded_file = st.file_uploader("Upload Image", type=["jpg", "jpeg", "png"])

    if uploaded_file is not None and phone.isdigit() and len(phone) == 10:
        save_dir = "data"
        os.makedirs(save_dir, exist_ok=True)
        save_path = os.path.join(save_dir, f"{phone}.jpg")
        image = Image.open(uploaded_file)
        image.save(save_path, format="JPEG")
        st.success(f"✅ Image saved successfully to: {save_path}")
    elif uploaded_file is not None:
        st.warning("⚠️ Invalid phone number — please enter a valid 10-digit number.")

    # ---- Generate QR ----
    if st.button("Generate QR Code"):
        if all([name, phone, Eme_contact1, Eme_contact2, Eme_contact3, blood_group, medical, ad, insurance, dob]) and uploaded_file is not None:
            patient_info = {
                "Name": name,
                "Phone": phone,
                "Emergency_contact1": Eme_contact1,
                "Emergency_contact2": Eme_contact2,
                "Emergency_contact3": Eme_contact3,
                "Blood_group": blood_group,
                "Medical_condition": medical,
                "Aadhaar": ad,
                "Insurance": insurance,
                "DOB": dob
            }

            data = json.dumps(patient_info)
            qr = qrcode.make(data)
            qr.save("patient_qr.png")

            st.markdown("<div class='qr-container'>", unsafe_allow_html=True)
            st.image("patient_qr.png", caption=f"{name}'s QR Code", use_container_width=False)
            st.markdown("</div>", unsafe_allow_html=True)

            with open("patient_qr.png", "rb") as file:
                st.download_button(
                    label="⬇️ Download QR Code",
                    data=file,
                    file_name=f"{name}_QR.png",
                    mime="image/png"
                )
        else:
            st.warning("⚠️ Please fill in all fields and upload an image before generating the QR Code.")


In [ ]:
import streamlit as st
import qrcode
import json
import os
from PIL import Image

def add_info():
    # ---- Custom CSS ----
    st.markdown("""
    <style>
    /* === GLOBAL BACKGROUND === */
    body {
        background: radial-gradient(circle at top left, #0d0d0d, #1a0000, #330000);
        color: white;
        font-family: 'Poppins', sans-serif;
    }

    /* === TITLE === */
    .main-title {
        text-align: center;
        font-size: 2.8rem;
        color: #ff073a;
        text-shadow: 0 0 25px #ff073a, 0 0 60px #ff073a;
        font-weight: 800;
        margin-top: 10px;
        animation: titlePulse 2s ease-in-out infinite alternate;
    }

    @keyframes titlePulse {
        from { text-shadow: 0 0 15px #ff073a; }
        to { text-shadow: 0 0 45px #ff073a, 0 0 90px #ff073a; }
    }

    /* === HEARTBEAT LINE === */
    .heartbeat-line {
        position: relative;
        width: 100%;
        height: 2px;
        background: linear-gradient(90deg, transparent, #ff073a, transparent);
        overflow: hidden;
        margin: 25px 0;
    }

    .heartbeat-line::before {
        content: '';
        position: absolute;
        left: -10%;
        width: 20%;
        height: 100%;
        background: #ff073a;
        animation: heartbeatMove 2.5s infinite ease-in-out;
    }

    @keyframes heartbeatMove {
        0% { left: -20%; opacity: 0; }
        25% { left: 25%; opacity: 1; }
        50% { left: 50%; opacity: 0.8; }
        75% { left: 75%; opacity: 1; }
        100% { left: 120%; opacity: 0; }
    }

    /* === INPUT FIELD GLOW === */
    .stTextInput > div > div > input {
        background-color: rgba(255, 7, 58, 0.08);
        border: 2px solid rgba(255, 7, 58, 0.5);
        border-radius: 14px;
        padding: 12px 18px;
        color: #fff;
        font-size: 1rem;
        font-weight: 500;
        box-shadow: 0 0 8px rgba(255, 7, 58, 0.2);
        transition: all 0.4s ease;
        backdrop-filter: blur(6px);
        animation: subtlePulse 3s infinite ease-in-out;
    }

    @keyframes subtlePulse {
        0%, 100% { box-shadow: 0 0 10px rgba(255, 7, 58, 0.3); }
        50% { box-shadow: 0 0 25px rgba(255, 7, 58, 0.7); }
    }

    .stTextInput > div > div > input:focus {
        border-color: #ff073a;
        box-shadow: 0 0 30px #ff073a, inset 0 0 15px rgba(255, 7, 58, 0.6);
        background-color: rgba(255, 7, 58, 0.12);
        transform: scale(1.03);
        transition: all 0.3s ease;
    }

    /* === FLOATING LABELS (KEY TAG STYLE) === */
    .stTextInput label {
        position: relative;
        color: #ff4f7a !important;
        font-weight: 700;
        font-size: 1.05rem;
        letter-spacing: 0.5px;
        text-shadow: 0 0 10px #ff073a, 0 0 25px rgba(255, 7, 58, 0.7);
        animation: glowLabel 2s ease-in-out infinite alternate;
        background: rgba(255, 7, 58, 0.05);
        padding: 4px 12px;
        border-left: 3px solid #ff073a;
        border-radius: 6px;
        display: inline-block;
        margin-bottom: 4px;
    }

    @keyframes glowLabel {
        from { text-shadow: 0 0 10px #ff073a; }
        to { text-shadow: 0 0 25px #ff5075, 0 0 40px #ff073a; }
    }

    /* === FILE UPLOADER === */
    .stFileUploader {
        border-radius: 15px;
        background: rgba(255, 7, 58, 0.06);
        padding: 20px;
        box-shadow: 0 0 20px rgba(255, 7, 58, 0.2);
        transition: all 0.3s ease;
        border: 1.5px dashed rgba(255, 7, 58, 0.6);
    }

    .stFileUploader:hover {
        box-shadow: 0 0 40px rgba(255, 7, 58, 0.8);
        transform: scale(1.02);
        border-color: #ff073a;
    }

    /* === BUTTONS === */
    .stButton > button {
        background: linear-gradient(90deg, #ff073a, #ff4f7a);
        color: white;
        font-weight: 600;
        padding: 12px 28px;
        border-radius: 10px;
        font-size: 1.1rem;
        border: none;
        box-shadow: 0 0 25px rgba(255, 7, 58, 0.7);
        transition: all 0.3s ease;
    }

    .stButton > button:hover {
        transform: scale(1.08);
        background: linear-gradient(90deg, #ff5075, #ff073a);
        box-shadow: 0 0 45px rgba(255, 7, 58, 0.9);
    }

    /* === QR CODE DISPLAY === */
    .qr-container {
        text-align: center;
        margin-top: 30px;
        animation: fadeIn 1.5s ease;
    }

    .qr-container img {
        border-radius: 12px;
        box-shadow: 0 0 35px #ff073a;
        animation: pulseQR 2s ease-in-out infinite alternate;
    }

    @keyframes pulseQR {
        from { box-shadow: 0 0 25px #ff073a; }
        to { box-shadow: 0 0 65px #ff073a; }
    }
    </style>
    """, unsafe_allow_html=True)

    # ---- HEADER ----
    st.markdown("<h1 class='main-title'>🩺 Life Saving QR Code Generator</h1>", unsafe_allow_html=True)
    st.markdown("<div class='heartbeat-line'></div>", unsafe_allow_html=True)

    # ---- INPUTS ----
    name = st.text_input("Name", placeholder="e.g. Prashanth")
    phone = st.text_input("User Phone Number", placeholder="e.g. 9876543210", max_chars=10)
    Eme_contact1 = st.text_input("Emergency Contact 1", placeholder="e.g. 9876543210", max_chars=10)
    Eme_contact2 = st.text_input("Emergency Contact 2", placeholder="e.g. 9876543210", max_chars=10)
    Eme_contact3 = st.text_input("Emergency Contact 3", placeholder="e.g. 9876543210", max_chars=10)
    blood_group = st.text_input("Blood Group", placeholder="e.g. B+")
    medical = st.text_input("Medical Condition (if any)", placeholder="e.g. Diabetes", value="NO")
    ad = st.text_input("Aadhaar Number", placeholder="e.g. 6555 xxxx xxxx 2412")
    insurance = st.text_input("Insurance Number", placeholder="e.g. 890xxxxxxxx", value="NO")
    dob = st.text_input("Date of Birth", placeholder="e.g. 15/09/2004")

    uploaded_file = st.file_uploader("Upload Image", type=["jpg", "jpeg", "png"])

    if uploaded_file is not None and phone.isdigit() and len(phone) == 10:
        save_dir = "data"
        os.makedirs(save_dir, exist_ok=True)
        save_path = os.path.join(save_dir, f"{phone}.jpg")
        image = Image.open(uploaded_file)
        image.save(save_path, format="JPEG")
        st.success(f"✅ Image saved successfully to: {save_path}")
    elif uploaded_file is not None:
        st.warning("⚠️ Invalid phone number — please enter a valid 10-digit number.")

    # ---- GENERATE QR ----
    if st.button("Generate QR Code"):
        if all([name, phone, Eme_contact1, Eme_contact2, Eme_contact3, blood_group, medical, ad, insurance, dob]) and uploaded_file is not None:
            patient_info = {
                "Name": name,
                "Phone": phone,
                "Emergency_contact1": Eme_contact1,
                "Emergency_contact2": Eme_contact2,
                "Emergency_contact3": Eme_contact3,
                "Blood_group": blood_group,
                "Medical_condition": medical,
                "Aadhaar": ad,
                "Insurance": insurance,
                "DOB": dob
            }

            data = json.dumps(patient_info)
            qr = qrcode.make(data)
            qr.save("patient_qr.png")

            st.markdown("<div class='qr-container'>", unsafe_allow_html=True)
            st.image("patient_qr.png", caption=f"{name}'s QR Code", use_container_width=False)
            st.markdown("</div>", unsafe_allow_html=True)

            with open("patient_qr.png", "rb") as file:
                st.download_button(
                    label="⬇️ Download QR Code",
                    data=file,
                    file_name=f"{name}_QR.png",
                    mime="image/png"
                )
        else:
            st.warning("⚠️ Please fill in all fields and upload an image before generating the QR Code.")


In [ ]:
import streamlit as st
import qrcode
import json
import os
from PIL import Image

def add_info():
    # ---- Custom CSS ----
    st.markdown("""
    <style>
    /* === GLOBAL BACKGROUND === */
    body {
        background: radial-gradient(circle at top left, #0d0d0d, #1a0000, #330000);
        color: white;
        font-family: 'Poppins', sans-serif;
    }

    /* === TITLE === */
    .main-title {
        text-align: center;
        font-size: 2.8rem;
        color: #ff073a;
        text-shadow: 0 0 25px #ff073a, 0 0 60px #ff073a;
        font-weight: 800;
        margin-top: 10px;
        animation: titlePulse 2s ease-in-out infinite alternate;
    }

    @keyframes titlePulse {
        from { text-shadow: 0 0 15px #ff073a; }
        to { text-shadow: 0 0 45px #ff073a, 0 0 90px #ff073a; }
    }

    /* === HEARTBEAT LINE === */
    .heartbeat-line {
        position: relative;
        width: 100%;
        height: 2px;
        background: linear-gradient(90deg, transparent, #ff073a, transparent);
        overflow: hidden;
        margin: 25px 0;
    }

    .heartbeat-line::before {
        content: '';
        position: absolute;
        left: -10%;
        width: 20%;
        height: 100%;
        background: #ff073a;
        animation: heartbeatMove 2.5s infinite ease-in-out;
    }

    @keyframes heartbeatMove {
        0% { left: -20%; opacity: 0; }
        25% { left: 25%; opacity: 1; }
        50% { left: 50%; opacity: 0.8; }
        75% { left: 75%; opacity: 1; }
        100% { left: 120%; opacity: 0; }
    }

    /* === INPUT FIELD LIGHTNING EFFECT === */
    .stTextInput > div > div > input {
        background-color: rgba(255, 7, 58, 0.08);
        border: 2px solid rgba(255, 7, 58, 0.6);
        border-radius: 14px;
        padding: 12px 18px;
        color: #fff;
        font-size: 1rem;
        font-weight: 600;
        box-shadow: 0 0 15px rgba(255, 7, 58, 0.3), inset 0 0 10px rgba(255, 7, 58, 0.1);
        backdrop-filter: blur(8px);
        animation: electricPulse 3s infinite linear;
        transition: all 0.4s ease;
    }

    @keyframes electricPulse {
        0% { box-shadow: 0 0 10px rgba(255, 7, 58, 0.4), inset 0 0 5px rgba(255, 7, 58, 0.2); }
        25% { box-shadow: 0 0 25px rgba(255, 7, 58, 0.7), inset 0 0 15px rgba(255, 7, 58, 0.3); }
        50% { box-shadow: 0 0 15px rgba(255, 7, 58, 0.6), inset 0 0 10px rgba(255, 7, 58, 0.3); }
        75% { box-shadow: 0 0 30px rgba(255, 7, 58, 0.8), inset 0 0 20px rgba(255, 7, 58, 0.4); }
        100% { box-shadow: 0 0 10px rgba(255, 7, 58, 0.4), inset 0 0 5px rgba(255, 7, 58, 0.2); }
    }

    .stTextInput > div > div > input:focus {
        border-color: #ff073a;
        background-color: rgba(255, 7, 58, 0.15);
        transform: scale(1.04);
        box-shadow: 0 0 40px #ff073a, inset 0 0 25px rgba(255, 7, 58, 0.6);
    }

    /* === BOLD LIGHTING LABEL (KEY TAG) === */
    .stTextInput label {
        position: relative;
        color: #ffffff !important;
        font-weight: 900;
        font-size: 1.1rem;
        letter-spacing: 1px;
        padding: 6px 14px;
        border-left: 4px solid #ff073a;
        border-radius: 6px;
        background: linear-gradient(90deg, rgba(255,7,58,0.15), transparent);
        text-transform: uppercase;
        text-shadow: 0 0 25px #ff073a, 0 0 45px #ff5075;
        animation: labelLightning 1.8s ease-in-out infinite alternate;
        margin-bottom: 5px;
        display: inline-block;
    }

    @keyframes labelLightning {
        from { text-shadow: 0 0 15px #ff073a, 0 0 35px #ff5075; }
        to { text-shadow: 0 0 45px #ff073a, 0 0 75px #ff5075; }
    }

    /* === FILE UPLOADER === */
    .stFileUploader {
        border-radius: 15px;
        background: rgba(255, 7, 58, 0.06);
        padding: 20px;
        box-shadow: 0 0 25px rgba(255, 7, 58, 0.3);
        transition: all 0.4s ease;
        border: 1.5px dashed rgba(255, 7, 58, 0.8);
        animation: flickerBox 2s infinite alternate;
    }

    @keyframes flickerBox {
        from { box-shadow: 0 0 20px rgba(255, 7, 58, 0.3); }
        to { box-shadow: 0 0 60px rgba(255, 7, 58, 0.8); }
    }

    .stFileUploader:hover {
        transform: scale(1.03);
        box-shadow: 0 0 70px rgba(255, 7, 58, 0.9);
    }

    /* === BUTTONS === */
    .stButton > button {
        background: linear-gradient(90deg, #ff073a, #ff4f7a);
        color: white;
        font-weight: 700;
        padding: 12px 28px;
        border-radius: 10px;
        font-size: 1.1rem;
        border: none;
        box-shadow: 0 0 35px rgba(255, 7, 58, 0.8);
        transition: all 0.3s ease;
    }

    .stButton > button:hover {
        transform: scale(1.08);
        background: linear-gradient(90deg, #ff5075, #ff073a);
        box-shadow: 0 0 55px rgba(255, 7, 58, 0.9);
    }

    /* === QR CODE DISPLAY === */
    .qr-container {
        text-align: center;
        margin-top: 30px;
        animation: fadeIn 1.5s ease;
    }

    .qr-container img {
        border-radius: 12px;
        box-shadow: 0 0 45px #ff073a;
        animation: pulseQR 2s ease-in-out infinite alternate;
    }

    @keyframes pulseQR {
        from { box-shadow: 0 0 30px #ff073a; }
        to { box-shadow: 0 0 70px #ff073a; }
    }
    </style>
    """, unsafe_allow_html=True)

    # ---- HEADER ----
    st.markdown("<h1 class='main-title'>🩺 Life Saving QR Code Generator</h1>", unsafe_allow_html=True)
    st.markdown("<div class='heartbeat-line'></div>", unsafe_allow_html=True)

    # ---- INPUTS ----
    name = st.text_input("Name", placeholder="e.g. Prashanth")
    phone = st.text_input("User Phone Number", placeholder="e.g. 9876543210", max_chars=10)
    Eme_contact1 = st.text_input("Emergency Contact 1", placeholder="e.g. 9876543210", max_chars=10)
    Eme_contact2 = st.text_input("Emergency Contact 2", placeholder="e.g. 9876543210", max_chars=10)
    Eme_contact3 = st.text_input("Emergency Contact 3", placeholder="e.g. 9876543210", max_chars=10)
    blood_group = st.text_input("Blood Group", placeholder="e.g. B+")
    medical = st.text_input("Medical Condition (if any)", placeholder="e.g. Diabetes", value="NO")
    ad = st.text_input("Aadhaar Number", placeholder="e.g. 6555 xxxx xxxx 2412")
    insurance = st.text_input("Insurance Number", placeholder="e.g. 890xxxxxxxx", value="NO")
    dob = st.text_input("Date of Birth", placeholder="e.g. 15/09/2004")

    uploaded_file = st.file_uploader("Upload Image", type=["jpg", "jpeg", "png"])

    if uploaded_file is not None and phone.isdigit() and len(phone) == 10:
        save_dir = "data"
        os.makedirs(save_dir, exist_ok=True)
        save_path = os.path.join(save_dir, f"{phone}.jpg")
        image = Image.open(uploaded_file)
        image.save(save_path, format="JPEG")
        st.success(f"✅ Image saved successfully to: {save_path}")
    elif uploaded_file is not None:
        st.warning("⚠️ Invalid phone number — please enter a valid 10-digit number.")

    # ---- GENERATE QR ----
    if st.button("Generate QR Code"):
        if all([name, phone, Eme_contact1, Eme_contact2, Eme_contact3, blood_group, medical, ad, insurance, dob]) and uploaded_file is not None:
            patient_info = {
                "Name": name,
                "Phone": phone,
                "Emergency_contact1": Eme_contact1,
                "Emergency_contact2": Eme_contact2,
                "Emergency_contact3": Eme_contact3,
                "Blood_group": blood_group,
                "Medical_condition": medical,
                "Aadhaar": ad,
                "Insurance": insurance,
                "DOB": dob
            }

            data = json.dumps(patient_info)
            qr = qrcode.make(data)
            qr.save("patient_qr.png")

            st.markdown("<div class='qr-container'>", unsafe_allow_html=True)
            st.image("patient_qr.png", caption=f"{name}'s QR Code", use_container_width=False)
            st.markdown("</div>", unsafe_allow_html=True)

            with open("patient_qr.png", "rb") as file:
                st.download_button(
                    label="⬇️ Download QR Code",
                    data=file,
                    file_name=f"{name}_QR.png",
                    mime="image/png"
                )
        else:
            st.warning("⚠️ Please fill in all fields and upload an image before generating the QR Code.")


In [ ]:
import streamlit as st
from PIL import Image
import json
from pyzbar.pyzbar import decode
import cv2
import numpy as np
import json
import os
from decode import decode_img

#st.set_page_config(page_title="📸 Simple Camera")





def scan():
# Streamlit's built-in camera input works on phone too!
    img_file = st.camera_input("Click below to take a photo")

    if img_file:
        img = Image.open(img_file)
        #st.image(img, caption="📷 Captured Photo", use_container_width=True)
        st.success("✅ Photo captured successfully!")
        img_path = "test.png"
        img.save(img_path)
        decoded=decode_img()
        print("Decoded Data:", decoded)
        return decoded



# --- Page 2: Result Page ---


In [ ]:
elif st.session_state.page == "result":
    import json
    from PIL import Image

    st.title("Person Details")

    st.markdown("""
    <style>
        .info-box {
            background: linear-gradient(145deg, #111827, #1e293b);
            color: #ffffff;
            padding: 20px 25px;
            border-radius: 12px;
            margin-bottom: 15px;
            box-shadow: 0 0 15px rgba(59,130,246,0.3);
            transition: transform 0.3s ease, box-shadow 0.3s ease;
        }

        .info-box:hover {
            transform: scale(1.02);
            box-shadow: 0 0 25px rgba(59,130,246,0.6);
        }

        .info-title {
            font-size: 18px;
            font-weight: bold;
            color: #60a5fa;  /* Bright blue for labels */
            margin-bottom: 6px;
            text-transform: uppercase;
            letter-spacing: 0.8px;
        }
                
        .info-title {
            color: #00f5ff;
            font-weight: bold;
            font-size: 18px;
            text-shadow: 0 0 8px #00f5ff, 0 0 15px #00f5ff;
        }



        .info-value {
            font-size: 16px;
            color: #fbbf24;  /* Gold-yellow for values */
            font-weight: 500;
            letter-spacing: 0.5px;
        }
    </style>
    """, unsafe_allow_html=True)


    decoded = st.session_state.get("decoded_data", None)
    if decoded is not None:
        try:
            data = json.loads(decoded)
        except Exception as e:
            st.error(f"Error reading QR data: {e}")
            st.stop()

        # ---- Patient Info Dictionary ----
        patient_info = {
            "Name": data.get("Name", "N/A"),
            "Emergency Contact 1": data.get("Emergency_contact1", "N/A"),
            "Emergency Contact 2": data.get("Emergency_contact2", "N/A"),
            "Emergency Contact 3": data.get("Emergency_contact3", "N/A"),
            "Blood Group": data.get("Blood_group", "N/A"),
            "Medical Condition": data.get("Medical_condition", "N/A"),
            "Aadhaar": data.get("Aadhaar", "N/A"),
            "Insurance": data.get("Insurance", "N/A"),
            "DOB": data.get("DOB", "N/A")
        }

        # ---- Display Image ----
        image_path = f"data/{data.get('Phone', 'unknown')}.jpg"
        try:
            image = Image.open(image_path)
            st.markdown('<div class="center-img">', unsafe_allow_html=True)
            st.image(image, caption=f"{data.get('Name', 'Patient')}", width=250)
            st.markdown('</div>', unsafe_allow_html=True)
        except FileNotFoundError:
            st.warning("⚠️ Image not found for this patient.")

        # ---- Display Patient Info Boxes ----
        st.markdown('<div style="display: flex; flex-wrap: wrap; gap: 20px;">', unsafe_allow_html=True)
        for key, value in patient_info.items():
            st.markdown(f"""
                <div class="info-box" style="flex: 1 1 45%;">
                    <div class="info-title">{key}</div>
                    <div class="info-value">{value}</div>
                </div>
            """, unsafe_allow_html=True)
        st.markdown('</div>', unsafe_allow_html=True)

In [ ]:
elif st.session_state.page == "result":
    import json
    from PIL import Image

    st.title("Person Details")

    # --- Heartbeat Red Theme CSS + Existing Blue Info Box CSS ---
    st.markdown("""
    <style>
        /* === GLOBAL BACKGROUND === */
        body {
            background: radial-gradient(circle at top, #0a0000, #1a0000, #330000, #0d0d0d);
            color: #ffffff;
            font-family: 'Poppins', sans-serif;
        }

        /* === PAGE TITLE === */
        h1, h2, h3 {
            color: #ff4d4d;
            text-shadow: 0 0 15px #ff0000;
            animation: heartbeat 2s infinite;
        }

        /* === INFO BOXES === */
        .info-box {
            background: linear-gradient(145deg, #1a0000, #330000);
            color: #ffffff;
            padding: 20px 25px;
            border-radius: 12px;
            margin-bottom: 15px;
            border: 1px solid rgba(255, 0, 0, 0.5);
            box-shadow: 0 0 20px rgba(255, 0, 0, 0.3);
            transition: transform 0.3s ease, box-shadow 0.3s ease;
        }

        .info-box:hover {
            transform: scale(1.02);
            box-shadow: 0 0 35px rgba(255, 0, 0, 0.6);
        }

        /* === TITLES === */
        .info-title {
            font-size: 18px;
            font-weight: bold;
            color: #ff6666;
            text-shadow: 0 0 8px #ff1a1a, 0 0 15px #ff3333;
            margin-bottom: 6px;
            text-transform: uppercase;
            letter-spacing: 0.8px;
        }

        /* === VALUES === */
        .info-value {
            font-size: 16px;
            color: #ffcc66;
            font-weight: 500;
            letter-spacing: 0.5px;
        }

        /* === CENTER IMAGE === */
        .center-img {
            display: flex;
            justify-content: center;
            margin-bottom: 25px;
            animation: pulseGlow 2s infinite;
        }

        /* === ANIMATIONS === */
        @keyframes heartbeat {
            0% { transform: scale(1); }
            25% { transform: scale(1.03); }
            50% { transform: scale(1); }
            75% { transform: scale(1.03); }
            100% { transform: scale(1); }
        }

        @keyframes pulseGlow {
            0% { filter: drop-shadow(0 0 10px rgba(255, 0, 0, 0.3)); }
            50% { filter: drop-shadow(0 0 25px rgba(255, 0, 0, 0.7)); }
            100% { filter: drop-shadow(0 0 10px rgba(255, 0, 0, 0.3)); }
        }
    </style>
    """, unsafe_allow_html=True)


    decoded = st.session_state.get("decoded_data", None)
    if decoded is not None:
        try:
            data = json.loads(decoded)
        except Exception as e:
            st.error(f"Error reading QR data: {e}")
            st.stop()

        # ---- Patient Info Dictionary ----
        patient_info = {
            "Name": data.get("Name", "N/A"),
            "Emergency Contact 1": data.get("Emergency_contact1", "N/A"),
            "Emergency Contact 2": data.get("Emergency_contact2", "N/A"),
            "Emergency Contact 3": data.get("Emergency_contact3", "N/A"),
            "Blood Group": data.get("Blood_group", "N/A"),
            "Medical Condition": data.get("Medical_condition", "N/A"),
            "Aadhaar": data.get("Aadhaar", "N/A"),
            "Insurance": data.get("Insurance", "N/A"),
            "DOB": data.get("DOB", "N/A")
        }

        # ---- Display Image ----
        image_path = f"data/{data.get('Phone', 'unknown')}.jpg"
        try:
            image = Image.open(image_path)
            st.markdown('<div class="center-img">', unsafe_allow_html=True)
            st.image(image, caption=f"{data.get('Name', 'Patient')}", width=250)
            st.markdown('</div>', unsafe_allow_html=True)
        except FileNotFoundError:
            st.warning("⚠️ Image not found for this patient.")

        # ---- Display Patient Info Boxes ----
        st.markdown('<div style="display: flex; flex-wrap: wrap; gap: 20px;">', unsafe_allow_html=True)
        for key, value in patient_info.items():
            st.markdown(f"""
                <div class="info-box" style="flex: 1 1 45%;">
                    <div class="info-title">{key}</div>
                    <div class="info-value">{value}</div>
                </div>
            """, unsafe_allow_html=True)
        st.markdown('</div>', unsafe_allow_html=True)


In [ ]:
print("prashmane")